In [16]:
#normalizar os arquivos de teste

import pandas as pd
import os

raiz = "large_scale"

meta = "large_scale_trated"


all_files = sorted([os.path.join(root, f) for root, _, files in os.walk(raiz) for f in files])
for i in range(0, len(all_files),2):
    info_file = all_files[i]
    items_file = all_files[i+1]

    info_df = pd.read_csv(info_file, header=None)
    for index, row in info_df.iterrows():
        if row[0] == 'n':
            n = int(row[1])
        elif row[0] == 'c':
            W = int(row[1])

    # Processa o arquivo items correspondente
    items_df = pd.read_csv(items_file)[[' price', ' weight']]
        
    # Adiciona n e W como primeira linha
    modified_df = pd.concat([
        pd.DataFrame([[n, W]], columns=items_df.columns),
        items_df
    ], ignore_index=True)

    base_name = os.path.basename(items_file).split('.')[0]  # Remove extensão
    output_file = os.path.join(meta, f"{base_name}.txt")
        
        # Salva como TXT com espaços
    with open(output_file, 'w') as f:
        for _, row in modified_df.iterrows():
            f.write(f"{int(row[0])} {int(row[1])}\n")
    
    print(f"✓ Arquivo gerado: {os.path.basename(output_file)}")

✓ Arquivo gerado: knapPI_14_200_1000_1_items.txt
✓ Arquivo gerado: knapPI_1_500_1000_1_items.txt
✓ Arquivo gerado: knapPI_2_500_1000_1_items.txt
✓ Arquivo gerado: knapPI_3_500_1000_1_items.txt


In [ ]:
import pandas as pd
import re

def parse_file(path, name):
    data = []
    current_test = {}  # Armazenará os dados completos de um teste (BB + Aprox)
    last_method = None
    
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            
            # Parse Número de Total de Itens e Peso máximo
            if line.startswith('* Número de itens: '):
                # Se já temos dados do método anterior, prepara para novo teste
                if last_method == '2-aproximativo':
                    data.append(current_test)
                    current_test = {}
                
                current_test['N'] = line.split(': ')[1].strip()
            

            elif line.startswith('* Peso médio:'):
                current_test['Peso_avg'] = line.split(': ')[1].strip()
            elif line.startswith('* Peso máximo:'):
                current_test['Peso_max'] = line.split(': ')[1].strip()
            elif line.startswith('* Valor médio:'):
                current_test['Valor_avg'] = line.split(': ')[1].strip()
            elif line.startswith('* Valor máximo:'):
                current_test['Valor_max'] = line.split(': ')[1].strip()
            elif line.startswith('* Capacidade da Mochila:'):
                current_test['W'] = line.split(': ')[1].strip()

            # Parse Branch and Bound value
            elif line.startswith('Branch and Bound:'):
                current_test['BB_sol'] = line.split(': ')[1].strip()
                last_method = 'BB'
            # Parse 2-aproximativo value
            elif line.startswith('2-aproximativo:'):
                current_test['Aprox_sol'] = line.split(': ')[1].strip()
                last_method = '2-aproximativo'
            # Parse Branch and Bound time
            elif line.startswith('Tempo Branch and Bound :'):
                current_test['BB_time'] = line.split(': ')[1].strip()
            # Parse Aproximativo time
            elif line.startswith('Tempo Aproximativo :'):
                current_test['Aprox_time'] = line.split(': ')[1].strip()
            # Parse RSS memory for BB
            elif line.startswith('* Memória residente real (RSS):') and '[BB]' in current_test.get('last_section', ''):
                current_test['BB_mem'] = line.split(': ')[1].replace(' KB', '').strip()
            # Parse RSS memory for aprox
            elif line.startswith('* Memória residente real (RSS):') and '[aprox]' in current_test.get('last_section', ''):
                current_test['Aprox_mem'] = line.split(': ')[1].replace(' KB', '').strip()
                current_test['Type'] = name
            # Track current section
            elif line.startswith('=== Métricas de Memória'):
                current_test['last_section'] = line
    
    # Add the last test if exists
    if current_test:
        data.append(current_test)
    
    return pd.DataFrame(data)

# Process both files
df_large = parse_file("large_results.txt", "Large")
df_low = parse_file("low_results.txt", "Low")

# Combine both DataFrames
df_combined = pd.concat([df_large, df_low], ignore_index=True)

# Convert scientific notation to normal numbers and clean data
def clean_value(val):
    if not isinstance(val, str):
        return val
    if 'e+' in val:
        return str(float(val))
    return val.replace(' KB', '')

df_combined = df_combined.applymap(clean_value)

# Reorder columns
columns = ['Type', 'N', 'W','Peso_avg','Peso_max','Valor_avg','Valor_max','Sol BB', 'Tempo BB', 'Custo BB', 
           'Sol 2-Aprox', 'Tempo 2-Aprox', 'Custo 2-Aprox']
df_combined = df_combined[columns]

# Fill empty values with empty string for better CSV output
df_combined = df_combined.fillna('')

# Save to CSV
df_combined.to_csv('resultados_combinados.csv', index=False)

print("DataFrame consolidado:")
print(df_combined)

DataFrame consolidado:
     Tipo    N      W Peso_avg Peso_max Valor_avg Valor_max   Sol BB Tempo BB  \
0   Large  200    997   498.09      997    711.59      1230     5397     1586   
1   Large  500   2543  513.864      997   494.142       998    28857   804097   
2   Large  500   2543  513.864      997   516.634      1074     4566    45777   
3   Large  500   2517  508.612      998   608.612      1098     7117   741009   
4     Low   20    879     54.9       96      54.3        91     1025     2657   
5     Low   10    269     53.9       95      41.2        87      295       25   
6     Low   20    878     54.9       96     54.25        91     1024     2550   
7     Low    4     20     6.75        9        12        15       35        9   
8     Low    4     11     4.75        7     10.25        13       23       12   
9     Low   15    375  49.4611  89.5962   37.5331   98.8525  481.069       38   
10    Low   10     60       13       30      10.5        20       52      111   
11   